# Core Sensitivity Testing

## Overview

This notebook demonstrates **processor core sensitivity analysis** for HEC-RAS 2D unsteady flow models. Understanding how computational performance scales with available CPU cores is essential for:

- **Resource Planning**: Determining optimal hardware for production runs
- **Cost-Benefit Analysis**: Evaluating whether additional cores justify computation time savings
- **Workflow Optimization**: Identifying diminishing returns for parallelization

### What This Notebook Does

1. Extracts a 2D example project (BaldEagleCrkMulti2D)
2. Runs the same plan with varying numbers of processor cores (1 to `NUM_WORKERS`)
3. Measures execution time for each configuration
4. Visualizes performance scaling and speedup factors
5. Identifies the optimal core count for this model

### LLM Forward Principle: Multi-Level Verifiability

This notebook produces:
- **CSV output**: Quantitative results for audit trail
- **Visualization**: Performance curves for visual inspection
- **Statistical metrics**: Speedup factors and efficiency percentages

### HEC-RAS Computational Background

HEC-RAS 2D uses **domain decomposition** to parallelize unsteady flow computations across multiple processor cores. Theoretical speedup is limited by:
- **Amdahl's Law**: Serial portions of the code limit parallelization benefits
- **Communication Overhead**: Inter-core data exchange increases with core count
- **Load Balancing**: Uneven mesh cell distribution can cause idle cores

**Expected Behavior**: Near-linear speedup for 2-4 cores, diminishing returns above 8 cores for most models.

### Reference

For HEC-RAS parallel computation details:
- [HEC-RAS User's Manual, Appendix E: Parallel Processing](https://www.hec.usace.army.mil/software/hec-ras/documentation.aspx)
- [HEC-RAS 2D Technical Reference: Domain Decomposition](https://www.hec.usace.army.mil/software/hec-ras/documentation.aspx)

For performance optimization best practices:
- [HEC-RAS Best Practices: 2D Model Development](https://www.hec.usace.army.mil/confluence/rasdocs/)


In [ ]:
# =============================================================================
# DEVELOPMENT MODE TOGGLE
# =============================================================================
USE_LOCAL_SOURCE = False  # <-- TOGGLE THIS

if USE_LOCAL_SOURCE:
    import sys
    from pathlib import Path
    local_path = str(Path.cwd().parent)
    if local_path not in sys.path:
        sys.path.insert(0, local_path)
    print(f"📁 LOCAL SOURCE MODE: Loading from {local_path}/ras_commander")
else:
    print("📦 PIP PACKAGE MODE: Loading installed ras-commander")

# Import ras-commander
from ras_commander import RasCmdr, RasExamples, RasGeo, RasPlan, init_ras_project, ras

# Additional imports
import h5py
import numpy as np
import requests
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import pyproj
from shapely.geometry import Point, LineString, Polygon
import xarray as xr
from pathlib import Path

# Verify which version loaded
import ras_commander
print(f"✓ Loaded: {ras_commander.__file__}")

## Parameters

Configure these values to customize the notebook for your project.

In [ ]:
# =============================================================================
# PARAMETERS - Edit these to customize the notebook
# =============================================================================
from pathlib import Path

# Project Configuration
PROJECT_NAME = "BaldEagleCrkMulti2D"  # Example project to extract
RAS_VERSION = "6.6"                   # HEC-RAS version

# Core Sensitivity Settings
PLAN = "03"                           # Plan to test (Plan 03 has 2D mesh)
NUM_WORKERS = 4                       # Maximum cores to test (will test 1 to NUM_WORKERS)


## Expected Results and Interpretation

### Performance Metrics

This analysis will generate the following metrics:

1. **Execution Time (seconds)**: Absolute wall-clock time for each core count
2. **Speedup Factor**: Ratio of 1-core time to N-core time (ideal = N)
3. **Parallel Efficiency**: Speedup / N cores × 100% (ideal = 100%)

### Typical Patterns

**Small 2D Models** (<10k cells):
- Minimal speedup beyond 2-4 cores
- Communication overhead dominates
- **Recommendation**: Use 2-4 cores

**Medium 2D Models** (10k-100k cells):
- Good speedup up to 4-8 cores
- Efficiency drops above 8 cores
- **Recommendation**: Use 4-8 cores

**Large 2D Models** (>100k cells):
- Linear speedup up to 8-16 cores
- Efficiency remains high for more cores
- **Recommendation**: Use 8-16 cores

### Common Pitfalls

- **Over-parallelization**: Using more cores than beneficial wastes resources
- **Hyperthreading**: Logical cores vs physical cores (use physical core count)
- **Shared Resources**: Other processes competing for CPU can skew results
- **Thermal Throttling**: Extended runs may trigger CPU frequency reduction

### Audit Trail Recommendations

Save the results CSV and plot to document:
- Hardware configuration (CPU model, RAM, OS)
- Model size (mesh cells, computation nodes, time steps)
- Core count vs execution time relationship
- Optimal core count for production runs


In [ ]:
import time
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from ras_commander import RasExamples, init_ras_project, RasCmdr, RasPlan, RasGeo

# Step 1: Initialize RasExamples and extract the BaldEagleCrkMulti2D project

project_path = RasExamples.extract_project(PROJECT_NAME, suffix="101")

# Create outputs subdirectory within project folder
outputs_dir = project_path / "outputs"
outputs_dir.mkdir(exist_ok=True)

# Step 2: Initialize the RAS Project Folder using init_ras_project (from ras_commander)
init_ras_project(project_path, RAS_VERSION)

# Step 3: Initialize a DataFrame to store execution results
results = []

# Step 4: Run sensitivity analysis for Plan 03 with core counts 1-8
plan_number = PLAN
print(f"Running sensitivity analysis for Plan {plan_number}")

# Clear geompre files before running the plan
plan_path = RasPlan.get_plan_path(plan_number)
RasGeo.clear_geompre_files(plan_path)

for cores in range(1, NUM_WORKERS + 1):
    print(f"Running with {cores} core(s)")
    # Set core count for this plan
    RasPlan.set_num_cores(plan_number, cores)
    
    # Time the execution of the plan
    start_time = time.time()
    RasCmdr.compute_plan(plan_number)
    execution_time = time.time() - start_time
    
    # Store the results
    results.append({
        "plan_number": plan_number,
        "cores": cores,
        "execution_time": execution_time
    })
    
    print(f"Execution time: {execution_time:.2f} seconds")

print("Sensitivity analysis complete")

# Step 5: Convert results into a DataFrame
results_df = pd.DataFrame(results)

# Save results to project outputs folder
results_csv_path = outputs_dir / "core_sensitivity_results.csv"
results_df.to_csv(results_csv_path, index=False)
print(f"Results saved to: {results_csv_path}")

NOTES FOR REVISIONS:
- Use HDF compute summary to show the time for each preprocesS/unsteady compute/postprocess step. 
- First, run preprocessor and then toggle options to only run unsteady compute and postprocess. 
- Plot each step separately. 



## Verification and Quality Checks

### Before Interpreting Results

Verify the following for valid sensitivity analysis:

1. **Model Convergence**: All runs completed successfully (no instabilities)
2. **Consistent Results**: Water surface elevations match across core counts (within numerical precision)
3. **Expected Timing**: 1-core run is slowest, higher core counts are faster
4. **No Outliers**: Execution times follow monotonic trend (no sudden jumps)

### Quality Assurance Steps

```python
# Check that all runs completed
assert len(results_df) == NUM_WORKERS, "Not all runs completed"

# Check that execution time decreases with more cores
assert results_df['execution_time_s'].is_monotonic_decreasing, "Execution time should decrease"

# Calculate speedup factors
results_df['speedup'] = results_df['execution_time_s'].iloc[0] / results_df['execution_time_s']
results_df['efficiency_pct'] = (results_df['speedup'] / results_df['num_cores']) * 100

print("Speedup Summary:")
print(results_df[['num_cores', 'execution_time_s', 'speedup', 'efficiency_pct']])
```

### Interpreting Efficiency

- **>90% efficiency**: Excellent scaling, model benefits from more cores
- **70-90% efficiency**: Good scaling, some overhead acceptable
- **50-70% efficiency**: Moderate scaling, consider diminishing returns
- **<50% efficiency**: Poor scaling, adding more cores not beneficial

### Documentation for Engineering Reports

Include in your report:
1. **Table**: Core count vs execution time vs speedup vs efficiency
2. **Plot**: Execution time curve and speedup curve
3. **Hardware specs**: CPU model, RAM, OS, HEC-RAS version
4. **Model specs**: Mesh cells, computation nodes, simulation duration
5. **Recommendation**: Optimal core count for production runs


In [ ]:
# Load the results from the project outputs folder
results_csv_path = project_path / "outputs" / "core_sensitivity_results.csv"
results_df = pd.read_csv(results_csv_path)

# Display the results dataframe for verification
print("results_df DataFrame (time is in seconds):")
display(results_df)

# Step 6: Calculate unit runtime (based on 1 core execution time)
results_df['unit_runtime'] = results_df.groupby('plan_number')['execution_time'].transform(lambda x: x / x.iloc[0])

# Get the project name from the ras object
project_name = ras.project_name

# Step 7: Plot a line chart for unit runtime vs. cores for each plan
plt.figure(figsize=(10, 6))
for plan in results_df['plan_number'].unique():
    plan_data = results_df[results_df['plan_number'] == plan]
    plt.plot(plan_data['cores'], plan_data['unit_runtime'], label=f"Plan {plan}")

plt.xlabel("Number of Cores")
plt.ylabel("Unit Runtime (Relative to 1 Core)")
plt.title(f"{project_name} (HEC Example Project)\nCore Count Sensitivity Analysis")
plt.legend(title="Plan Number")
plt.grid(False)
plt.vlines([1,2,3,4], ymin=0, ymax=1.2, linestyles='dotted', alpha=0.3)
plt.gca().xaxis.set_major_locator(plt.MultipleLocator(1))
plt.show()